In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
train_list =glob.glob('train*.csv')
test_list = glob.glob('test*.csv')

In [3]:
train = pd.concat([pd.read_csv(i)for i in train_list])
test = pd.concat([pd.read_csv(i)for i in test_list])

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [5]:
train['time']

0         2019-09-11 20:00:00
1         2019-09-11 20:00:01
2         2019-09-11 20:00:02
3         2019-09-11 20:00:03
4         2019-09-11 20:00:04
                 ...         
241195    2019-11-04 14:59:55
241196    2019-11-04 14:59:56
241197    2019-11-04 14:59:57
241198    2019-11-04 14:59:58
241199    2019-11-04 14:59:59
Name: time, Length: 550800, dtype: object

In [6]:
train_scaled = scaler.fit_transform(train.drop(['time'] , axis=1))
train_scaled = pd.DataFrame(train_scaled , columns=train.drop('time', axis=1).columns)

In [7]:
train_scaled['time'] = train['time'].tolist()

In [8]:
train_scaled['time']= pd.to_datetime(train_scaled['time'])

train.shape

In [9]:
import tensorflow as tf
from tensorflow import keras

In [10]:
print(tf.__version__)

2.2.0


In [11]:
from tqdm import tqdm

In [12]:
from tqdm.notebook import trange
from datetime import timedelta

import dateutil

valid_idxs = []
timestamp_list = train_scaled['time'].astype(str).tolist()
WINDOW_SIZE = 9

for L in trange(len(timestamp_list) - WINDOW_SIZE + 1):
    R = L + WINDOW_SIZE - 1
    if dateutil.parser.parse(timestamp_list[R]) - dateutil.parser.parse(timestamp_list[L]) == timedelta(seconds=WINDOW_SIZE - 1):
        valid_idxs.append(L)
    else :
        print(L, timestamp_list[L:R])

309592 ['2019-09-15 09:59:52', '2019-09-15 09:59:53', '2019-09-15 09:59:54', '2019-09-15 09:59:55', '2019-09-15 09:59:56', '2019-09-15 09:59:57', '2019-09-15 09:59:58', '2019-09-15 09:59:59']
309593 ['2019-09-15 09:59:53', '2019-09-15 09:59:54', '2019-09-15 09:59:55', '2019-09-15 09:59:56', '2019-09-15 09:59:57', '2019-09-15 09:59:58', '2019-09-15 09:59:59', '2019-11-01 20:00:00']
309594 ['2019-09-15 09:59:54', '2019-09-15 09:59:55', '2019-09-15 09:59:56', '2019-09-15 09:59:57', '2019-09-15 09:59:58', '2019-09-15 09:59:59', '2019-11-01 20:00:00', '2019-11-01 20:00:01']
309595 ['2019-09-15 09:59:55', '2019-09-15 09:59:56', '2019-09-15 09:59:57', '2019-09-15 09:59:58', '2019-09-15 09:59:59', '2019-11-01 20:00:00', '2019-11-01 20:00:01', '2019-11-01 20:00:02']
309596 ['2019-09-15 09:59:56', '2019-09-15 09:59:57', '2019-09-15 09:59:58', '2019-09-15 09:59:59', '2019-11-01 20:00:00', '2019-11-01 20:00:01', '2019-11-01 20:00:02', '2019-11-01 20:00:03']
309597 ['2019-09-15 09:59:57', '2019-09-

In [13]:
print(max(valid_idxs))
print(len(valid_idxs)-1)

550791
550783


In [14]:
set (list(range(0,550791+1))) - set(valid_idxs)

{309592, 309593, 309594, 309595, 309596, 309597, 309598, 309599}

In [15]:
train_scaled[309590:309600]['time']

309590   2019-09-15 09:59:50
309591   2019-09-15 09:59:51
309592   2019-09-15 09:59:52
309593   2019-09-15 09:59:53
309594   2019-09-15 09:59:54
309595   2019-09-15 09:59:55
309596   2019-09-15 09:59:56
309597   2019-09-15 09:59:57
309598   2019-09-15 09:59:58
309599   2019-09-15 09:59:59
Name: time, dtype: datetime64[ns]

In [16]:
tag_values = np.array(train_scaled.drop('time',axis=1), dtype=np.float32)

In [17]:
data_list = []
for i in valid_idxs :
    item = {} 
    last = i + WINDOW_SIZE - 1
    item["given"] = tag_values[i : last-1]
    item["answer"] = tag_values[last]
    data_list.append(item)

In [18]:
data_list[0]['given'][-1]

array([9.5041323e-01, 4.8464945e-01, 9.2348807e-02, 9.2435157e-01,
       8.7563485e-01, 0.0000000e+00, 4.4892412e-03, 7.5154954e-01,
       0.0000000e+00, 3.0276409e-04, 1.0000000e+00, 9.8799998e-01,
       4.7067997e-01, 4.7764438e-01, 4.0159157e-01, 6.1166805e-01,
       1.7618390e-03, 4.8344983e-03, 8.4990239e-01, 8.8116044e-01,
       5.7252520e-01, 5.9469765e-01, 2.6428846e-01, 1.6577971e-01,
       1.7335278e-01, 0.0000000e+00, 1.1757903e-01, 2.7608019e-01,
       1.8668300e-02, 5.9035099e-01, 3.9949328e-01, 5.9121621e-01,
       0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.0000000e+00,
       7.2916669e-01, 0.0000000e+00, 3.2947403e-01, 4.1381672e-01,
       5.7635611e-01, 6.9404519e-01, 7.9846829e-01, 6.2500000e-02,
       7.4573243e-01, 0.0000000e+00, 0.0000000e+00, 3.0887550e-01,
       5.0649351e-01, 7.9096335e-01, 7.3602962e-01, 0.0000000e+00,
       6.3928610e-01, 4.8246446e-01, 5.3360903e-01, 5.2754074e-01,
       9.9067557e-01, 2.8169015e-01, 9.9991953e-01, 0.0000000e

In [19]:
data_list = np.array(data_list)
np.save('train_preprocessed.npy' , data_list)

In [20]:
from tensorflow.keras import layers

In [21]:
class Encoder(layers.Layer):
    def __init__(self,name='encoder',latent_dim = 32 , intermediate_dim=64):
        super(Encoder, self).__init__()
        self.lstm_proj = layers.LSTM(intermediate_dim)
        self.dense_mean = layers.Dense(latent_dim)
        self.dense_log_var = layers.Dense(latent_dim)
        self.sampling = Sampling()
    
    def call(self, inputs):
        x = self.lstm_proj(inputs)
        z_mean = self.dense_mean(x)
        z_log_var = self.dense_log_var(x)
        z = self.sampling((z_mean, z_log_var))
        
        return z_mean, z_log_var, z

In [22]:
class Sampling(layers.Layer):
 
  def call(self, inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [ ]:
class Discriminator(layers.Layer) :
    def __init__(self, target=56, timestep=10, name='discriminator'):
        super(Discriminator , self).__init__()
        self.LSTM_discri = layers.LSTM(inputs)
        self.dense_result = layers.Dense(1,activation='softmax')
    
    def(self,inputs) :
        x= self.LSTM_discri(inputs)
        return self.dense_result(x)

In [ ]:
class Decoder(layers.Layer):
  
  def __init__(self, original_dim,intermediate_dim=64,name='decoder'):
    super(Decoder, self).__init__()
    self.dense_proj = layers.Dense(intermediate_dim, activation='relu')
    self.lstm_output = layers.LSTM(original_dim)

  def call(self, inputs):
    x = self.dense_proj(inputs)
    return self.lstm_output(x)